<a href="https://colab.research.google.com/github/DataParadox/iTrustBD/blob/main/Codes/iTrustBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Project from GitHub and Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/'

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works


In [ ]:
!git clone 'https://github.com/DataParadox/iTrustBD.git'

In [3]:
%cd iTrustBD

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/iTrustBD


In [ ]:
!git pull

remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 97 (delta 47), reused 4 (delta 4), pack-reused 15
Unpacking objects: 100% (97/97), done.
From https://github.com/DataParadox/iTrustBD
   357e37d..03523e1  main       -> origin/main
Updating 357e37d..03523e1
Fast-forward
 Codes/iTrustBD.ipynb    | 177 +++++++++++++++++++++++++++---------------------
 README.md               |  79 +++++++++++++++++++++
 _config.yml             |   2 +
 _layouts/default.html   |  10 +++
 img/classEqn.svg        |  96 ++++++++++++++++++++++++++
 img/network_preview.png | Bin 0 -> 427257 bytes
 6 files changed, 286 insertions(+), 78 deletions(-)
 create mode 100644 img/classEqn.svg
 create mode 100644 img/network_preview.png


# Set Datasets

In [4]:
def setDatasets(dataset):           # dataset -> Name of the dataset, i.e. 'soc-sign-bitcoinalpha-dataset' 
  import pandas as pd
  
  nodeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-nodes-table.csv')
  edgeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-edge-table.csv')
  nodeTable['behavior'] = nodeTable['Class']

  edgeTableTargetSource = edgeTable[['Target', 'Source']].astype(str)
  neighborTableIn = edgeTableTargetSource.groupby(['Target'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableIn.columns = neighborTableIn.columns.str.replace('Source', 'Neighbors')

  neighborTableOut = edgeTableTargetSource.groupby(['Source'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableOut.columns = neighborTableOut.columns.str.replace('Target', 'Neighbors')

  return nodeTable, edgeTable, neighborTableIn, neighborTableOut

# Generate Random Probability

In [5]:
def randProbability(min, max):
  # generate random floating point values
  import random
  return min + (random.random() * (max - min))

# Behavior Spread Based on Mutation Factor

In [6]:
def behaviorSpreadMutate(): #nodeTable
  import random
  meu_m = randProbability(0, 0.01)                                     # Random mutation factor generation in the range of (0, 0.01)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) > (1-meu_m):                           # If cross the threshold of meu_m
        nodeTable.at[trader, 'behavior'] =  random.choice([-1, 0, 1])   # Change behavior randomly       nodeTable.iloc[trader].behavior
  #return nodeTable 

# Behavior Spread Based on Neighbors

In [7]:
def behaviorSpreadNeighbors(): #nodeTable, neighborTableIn
  import random
  gamma = randProbability(0, 0.05)       # Random transmission rate generation in the range of (0, 0.05)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) > (1-gamma):
        try:
          random_neighbor = random.choice(list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == nodeTable.iloc[trader].Id.astype(int)]['Neighbors'].item().split(","))))
          #print(nodeTable.iloc[trader].Id.item(), random_neighbor, 'A', list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == nodeTable.iloc[trader].Id.astype(int)]['Neighbors'].item().split(","))))
          nodeTable.at[trader, 'behavior'] =  int(nodeTable.loc[nodeTable['Id'].astype(int) == int(random_neighbor)].behavior.to_string(index=False))
        except: #nodeTable.iloc[trader].behavior
          pass
  #return nodeTable

# Setting of Parameters

In [8]:
def setParameters():
  beta_T = randProbability(0, 1)
  beta_S = beta_T

  meu_T = randProbability(0, beta_T)
  meu_S = meu_T

  return beta_T, beta_S, meu_T, meu_S

# Calculating Propotion of Neighbors

In [9]:
def neighborProportion(nodeId): #nodeId, nodeTable, neighborTableIn, neighborTableOut
  setNeighbor = set(list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == int(nodeId)]['Neighbors'].item().split(","))))
  setNeighbor = set(list(map(int, neighborTableOut.loc[neighborTableOut['Source'].astype(int) == int(nodeId)]['Neighbors'].item().split(",")))).union(setNeighbor)

  countTrusty = 0
  countSuspecious = 0
  countNormal = 0

  for neighbor in setNeighbor:
    if int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 0:
      countNormal = countNormal + 1
    elif int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 1:
      countTrusty = countTrusty + 1
    else:
      countSuspecious = countSuspecious + 1
  
  return countTrusty/(countTrusty+countNormal+countSuspecious), countNormal/(countTrusty+countNormal+countSuspecious), countSuspecious/(countTrusty+countNormal+countSuspecious)
  #return sigma_T, sigma_N, sigma_S

# Susceptible-Infected-Susceptible (SIS) Model

In [10]:
def modelSIS(): #nodeTable, neighborTableIn, neighborTableOut
  beta_T, beta_S, meu_T, meu_S = setParameters()
  for index in nodeTable.index:
    try:
      sigma_T, sigma_N, sigma_S = neighborProportion(nodeTable.Id[index], nodeTable, neighborTableIn, neighborTableOut)
      if(nodeTable.Id[index].astype(int) == 0):
        if(nodeTable.behavior[index].astype(int) == 0):
          P_NT = beta_T*sigma_T
          P_NS = beta_S*sigma_S
          if((P_NT >= P_NS) and (randProbability(0, 1) > (1 - P_NT))):
            nodeTable.at[index, 'behavior'] = 1 #nodeTable.behavior[index] = 1
          elif((P_NT < P_NS) and (randProbability(0, 1) > (1 - P_NS))):
            nodeTable.at[index, 'behavior'] = -1 #nodeTable.behavior[index] = -1
        elif((nodeTable.behavior[index].astype(int) == 1) and (randProbability(0, 1) > (1 - meu_T))):
          nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
        elif((nodeTable.behavior[index].astype(int) == -1) and (randProbability(0, 1) > (1 - meu_S))):
          nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
    except:
      pass
  #return nodeTable

# Voter Model

In [11]:
def modelVoter(): #nodeTable, neighborTableIn, neighborTableOut
  beta_T, beta_S, meu_T, meu_S = setParameters()
  for index in nodeTable.index:
    try:
      sigma_T, sigma_N, sigma_S = neighborProportion(nodeTable.Id[index], nodeTable, neighborTableIn, neighborTableOut)
      if(nodeTable.Id[index].astype(int) == 0):
        if(nodeTable.behavior[index].astype(int) == 0):
          P_NT = sigma_T * meu_T
          P_NS = sigma_S * meu_S
          if((P_NT >= P_NS) and (randProbability(0, 1) > (1 - P_NT))):
            nodeTable.at[index, 'behavior'] = 1 #nodeTable.behavior[index] = 1
          elif((P_NT < P_NS) and (randProbability(0, 1) > (1 - P_NS))):
            nodeTable.at[index, 'behavior'] = -1 #nodeTable.behavior[index] = -1
        elif(nodeTable.behavior[index].astype(int) == 1):
          P_TN = sigma_S * beta_T
          if(randProbability(0, 1) > (1 - P_TN)):
            nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
        else:
          P_SN = sigma_S * beta_S
          if(randProbability(0, 1) > (1 - P_SN)):
            nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
    except:
      pass
  #return nodeTable

# Bilingual Model

In [12]:
def modelBilingual(): #nodeTable, neighborTableIn, neighborTableOut
  beta_T, beta_S, meu_T, meu_S = setParameters()
  for index in nodeTable.index:
    try:
      sigma_T, sigma_N, sigma_S = neighborProportion(nodeTable.Id[index]) #nodeTable, neighborTableIn, neighborTableOut
      if(nodeTable.Id[index].astype(int) == 0):
        if(nodeTable.behavior[index].astype(int) == 0):
          P_NT = (1 - sigma_S) * meu_T
          P_NS = (1 - sigma_T) * meu_S
          if((P_NT >= P_NS) and (randProbability(0, 1) > (1 - P_NT))):
            nodeTable.at[index, 'behavior'] = 1 #nodeTable.behavior[index] = 1
          elif((P_NT < P_NS) and (randProbability(0, 1) > (1 - P_NS))):
            nodeTable.at[index, 'behavior'] = -1 #nodeTable.behavior[index] = -1
        elif(nodeTable.behavior[index].astype(int) == 1):
          P_TN = sigma_S * beta_T
          if(randProbability(0, 1) > (1 - P_TN)):
            nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
        else:
          P_SN = sigma_T * beta_S
          if(randProbability(0, 1) > (1 - P_SN)):
            nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
    except:
      pass
  #return nodeTable

# Behavior Spread

In [13]:
def behaviorSpread(dataset, total_epoch):
  import pandas as pd
  import random
  global nodeTable, edgeTable, neighborTableIn, neighborTableOut
  nodeTable, edgeTable, neighborTableIn, neighborTableOut = setDatasets(dataset)
  filepath = '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/Datasets/Epoch_Out/node_table_{}.csv'.format(0)
  nodeTable.to_csv(filepath, index=False)
  epoch = 0
  while epoch < total_epoch:
    epoch = epoch + 1
    behaviorSpreadNeighbors()
    behaviorSpreadMutate()
    try:
      model = random.choice([1, 2, 3])
      if(model == 1):
        modelSIS()
      elif(model == 2):
        modelVoter()
      else:
        modelBilingual()
    except:
      pass
    filepath = '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/Datasets/Epoch_Out/node_table_{}.csv'.format(epoch)
    nodeTable.to_csv(filepath, index=False)

    #for i in neutral_traders:
      # Select a random neighbor j with probability proportional to in_degree.
      #if i neutral & j not neutral:
        # i can assume the state of j under the rules of the model
      #elif i not neutral:
          # set i neutral under the rules of the model

In [36]:
dataset = 'soc-sign-bitcoinalpha-dataset'
behaviorSpread(dataset, total_epoch=100)